# **ZincSight**: Interpretable prediction of zinc ion locations in proteins

Due to current Google Colab limitations, in case ZincSight crashes, please reset the runtime (`Runtime` → `Disconnect and delete runtime`) and try again. If the issue presists, contact mechtinger1@mail.tau.ac.il and include the protein IDs/structures that you used.

In [ ]:
#@title Initialization: Click `▶` and wait, then rerun this cell after runtime restarts {display-mode: "form"}
#@markdown 🔴 **Note:** The runtime will crash after the first run — this is expected due to how Colab handles `condacolab` installation.

import os
import sys
import time

def is_first_run():
    return 'condacolab' not in sys.modules

if is_first_run():
    print("🔄 Installing condacolab. Runtime will automatically restart shortly...")
    # Install condacolab
    !pip install -q condacolab
    import condacolab
    condacolab.install()  # This will force a runtime restart
else:
    print("✅ condacolab is already installed. Continuing...")

In [ ]:
#@title Query for ZincSight {display-mode: "form"}
#@markdown   **Enter your protein(s)** in the field below or upload **PDB** or **MMCIF** structures.
# @markdown - Query IDs can correspond to structures from **PDB** (can be either asymmetric units or biological assemblies), **AlphaFoldDB** (UniProt ID or AlphaFold model ID), or the **ESM Metagenomic Atlas**. The default field includes an example for each.
# @markdown - To upload your own structure files, check the appropriate box below and run this cell.
#@markdown ---
#@markdown <p></p>
structure_ids_for_download = '8QEP, 2A0S-assembly1, 1KLS, P0A6G5, AF-A0A068N621-F1-v4, MGYP002718891411' # @param {type:"string"}
upload_structures = False # @param {type:"boolean"}
if structure_ids_for_download.strip() == '' or upload_structures:
  !mkdir -p /content/query_structures
  %cd /content/query_structures
  from google.colab import files
  uploaded = files.upload()
  %cd /content
include_histidine_rotamers = True # @param {type: "boolean"}

In [ ]:
%%capture captured_output
#@title Execute ZincSight {display-mode: "form"}

if not os.path.exists("ENV_SET_UP"):
    #Setup environment
    !pip install biopython==1.79
    !conda install pymol-open-source=3.0.0 scipy matplotlib psycopg2-binary scikit-learn python-dotenv joblib aria2p -c conda-forge
    !sudo apt install pigz

    # Install Essential System Packages and GPG Tools
    !sudo apt install dirmngr ca-certificates software-properties-common gnupg gnupg2 apt-transport-https curl -y

    # Install PostgresSQL
    !curl -fSsL https://www.postgresql.org/media/keys/ACCC4CF8.asc | gpg --dearmor | sudo tee /usr/share/keyrings/postgresql.gpg > /dev/null
    !echo 'deb [arch=amd64,arm64,ppc64el signed-by=/usr/share/keyrings/postgresql.gpg] http://apt.postgresql.org/pub/repos/apt/ jammy-pgdg main' | sudo tee /etc/apt/sources.list.d/pgdg.list
    !sudo apt update
    !sudo apt install postgresql-client-15 postgresql-15 -y
    !sudo service postgresql start

    # Create a new PostgreSQL database if it does not already exist
    !sudo -u postgres psql -tc "SELECT 1 FROM pg_database WHERE datname = 'zincsight_pipeline_db';" | grep -q 1 || sudo -u postgres psql -c "CREATE DATABASE zincsight_pipeline_db;"
    !sudo -u postgres psql -c "GRANT ALL PRIVILEGES ON DATABASE zincsight_pipeline_db TO postgres;"
    !sudo -u postgres psql -c "ALTER USER postgres WITH PASSWORD 'postgres';"

    # Clone the GitHub repository and change into the repo directory
    !git clone https://github.com/MECHTI1/ZincSight.git
    %cd ZincSight/
    # Add essential SQL tables to the established database
    !sudo -u postgres psql -d zincsight_pipeline_db -P pager=off -f "src/setup_pg_db_with_tables/PostgreSQL_4_necessary_tables.sql"

    # Run ZincSight
    import os
    print("Current Path:", os.getcwd()) # Debugging: Print current working directory

    import sys
    sys.path.append('/content/ZincSight')

!touch ENV_SET_UP

from main_execute import execute_zincsight

# Define paths explicitly
query_structures_path, output_path  = '/content/query_structures', '/content/output' # Path to input structures

# Ensure paths exist
if not os.path.exists(query_structures_path):
    os.makedirs(query_structures_path)
if not os.path.exists(output_path):
    os.makedirs(output_path)  # Create output directory if it doesn't exist


# Call the function with explicit paths
output_compressed_file = execute_zincsight(include_histidine_rotamers, structure_ids_for_download, query_structures_path,output_path)

In [ ]:
#@title Download Results {display-mode: "form"}
if output_compressed_file!= False:
    from google.colab import files
    files.download(output_compressed_file)
else: print ("No predicted zinc-binding sites within the given query structures!")